# Data Manipulation

Let's go back to the `DATA` step and other techniques we can learn from it.

## Concatenating Datasets
Let's say you have two or more tables that have the same columns and you want to combine them all into one single table, or "concatenate" into a single table (think of the `rbind()` function in R. We can do that by using a simple `DATA` step.
```
DATA output-table;
    SET input-table1 input-table2 .. ;
RUN;
```

Let's take a look at an example below.

In [1]:
/* importing data for analysis */
proc import datafile="C:\Users\rdominguez\datasets\class_1.csv" dbms=csv 
		out=class;
run;

proc import datafile="C:\Users\rdominguez\datasets\class_2.csv" dbms=csv
    out=class2;
run;

title 'Class1 dataset';
proc contents data=class;
run;
title;

title 'Class2 dataset';
proc contents data=class2;
run;
title;

The code above imports 2 different `csv` files, both containing the same number of columns with the same name as shown from the proc contents output above. One dataset has 10 observations while the other has 32. Let's combine it all into one SAS table.

In [4]:
DATA class_all;
    set class class2;
run;

proc contents data=class_all;
run;

In the case, you have to rename columns to match you can use the following syntax below:
```
DATA output-table;
    SET input-table1(rename=(currentcolname = newname))
        input-table2. . . .;
RUN;
```

## Merging Datasets

Let's say you have two different datasets that you need to merge or join into a single table. You can do this two different ways, using `PROC SQL` or a merge statement within the `DATA` step. We will visit `PROC SQL` later.

Let's take a look at the class dataset we created in the previous example.

In [6]:
proc print data=class_all (obs=5);
run;

Obs,Name,Age,Weight,Height,Grade,GPA
1,Alex Smith,20,145.6,5.8,Junior,3.27
2,Jordan Brown,19,134.2,5.7,Freshman,2.89
3,Taylor Williams,21,180.3,6.1,Senior,3.61
4,Morgan Johnson,22,155.4,5.9,Junior,3.01
5,Jamie Davis,18,120.5,5.5,Freshman,2.75


In [10]:
/* import a dataset that contains their teachers name */
proc import datafile="C:\Users\rdominguez\datasets\class_teachers.csv" dbms=csv 
		out=teacher;
run;

proc print data=teacher (obs=5);
run;

Obs,Name,Grade,Teacher
1,Alex Smith,Junior,Ms. Foster
2,Jordan Brown,Freshman,Dr. Davis
3,Taylor Williams,Senior,Mr. Evans
4,Morgan Johnson,Junior,Ms. Clarke
5,Jamie Davis,Freshman,Mrs. Anderson


Now, the dataset we just imported contains the teacher's name for these students. We want to merge these datasets together so that they are all in one table. We can do that with this syntax below:
:::{admonition} *MERGE STATEMENT*
:class: dropdown
```
DATA outout-table;
    MERGE input-table1 input-table2;
    BY column(s);
RUN;
```
:::

Please note that the **merge** statement requires that all tables should be sorted by the key variables specifed by the merge due to the DATA step processing (compilation and execution) mechanisms. We see in the code below that the data step used to merge these tables are preceded by 2 proc steps that sort the `class` table and the `teachers` table. SAS compares the rows sequentially. You can also list multiple tables in the `MERGE` statement, as long as those tables have the common matching keys.

The default merge is typically a `One-to-One` merge.

In [12]:
/* sort class */
proc sort data=class_all;
    BY Name Grade;
run;

/* sort teacher */
proc sort data=teacher;
    By Name Grade;
run;

/* merge together */
data class_merge;
    merge class_all teacher;
    By Name Grade;
run;

/* print dataset to view */
proc print data=class_merge (obs=10);
run;

Obs,Name,Age,Weight,Height,Grade,GPA,Teacher
1,Alex Johnson,19,142.8,5.6,Freshman,2.95,
2,Alex Smith,20,145.6,5.8,Junior,3.27,Ms. Foster
3,Alexis Jones,18,125.4,5.5,Freshman,2.81,Mr. Lewis
4,Avery Thompson,20,147.3,5.7,Senior,3.5,Ms. Foster
5,Avery Wilson,21,169.5,6.1,Junior,3.42,Ms. Foster
6,Bailey White,20,163.5,6.1,Sophomore,3.38,Dr. Garcia
7,Blake Robinson,20,168.3,6.1,Sophomore,3.35,Ms. Foster
8,Cameron Davis,20,172.3,6,Sophomore,3.5,Mr. Brown
9,Carter Davis,20,165.4,6,Sophomore,3.33,Ms. Clarke
10,Casey Green,21,153.7,6,Senior,3.52,Mr. Lewis


Now let's say that for every student, they have multiple teachers. Ie, `Alex Smith` can have `Ms.Foster` or `Mr.Lewis` as a teacher, say for different classes. If this were the case, then the merge would be a `One-to-Many` merge. 

In the case there are rows in a table with no matching rows based off our `BY` statement, then the missing columns will remain as missing values when the new row is written to the merge.

## Summarizing Dataset
Understanding the data step processing steps is crucial in more complex data manipulation. Let's first talk about creating an accumulating column.

To do so, we will be importing a dataset that contains Guam temperature and precipitation levels for the month of September 2024.

In [31]:
proc import datafile="C:\Users\rdominguez\datasets\guam_temp.csv" dbms=csv 
		out=temp;
run;

proc print data=temp(obs=5);
run;

Obs,Date,Maximum,Minimum,Average,Departure,HDD,CDD,Precipitation,New Snow,Snow Depth
1,2024-09-01,88,77,82.5,7,0,18,0.07,0,0
2,2024-09-02,88,78,83,12,0,18,0.01,0,0
3,2024-09-03,89,77,83,12,0,18,0.21,0,0
4,2024-09-04,87,78,82.5,0.7,0,18,0.25,0,0
5,2024-09-05,81,76,78.5,-3.3,0,14,41,0,0


Let's say we want to add in a new column that accumulates the daily Precipitation. We know that the PDV iteratively reads in the rows one-by-one. In order to perform our accumulating calcuation To do so, we need to ensure that the PDV remembers the last value it creates before it re-initializes again.

In [108]:
data temp_modified;
    set temp;
    retain RainSum 0;
    RainSum=RainSum + Precipitation;
run;

proc print data=temp_modified (obs=6);
run;

Obs,Date,Maximum,Minimum,Average,Departure,HDD,CDD,Precipitation,New Snow,Snow Depth,RainSum
1,2024-09-01,88,77,82.5,7,0,18,0.07,0,0,0.07
2,2024-09-02,88,78,83,12,0,18,0.01,0,0,0.08
3,2024-09-03,89,77,83,12,0,18,0.21,0,0,0.29
4,2024-09-04,87,78,82.5,0.7,0,18,0.25,0,0,0.54
5,2024-09-05,81,76,78.5,-3.3,0,14,41,0,0,41.54
6,2024-09-06,89,78,83.5,17,0,19,0.08,0,0,41.62


:::{admonition} Code Breakdown:
:class: important

The code above creates a new dataset called `temp_modified` from the and is set to be the `temp` dataset. It creates a cumulative sum of the column `Precipitation`

- `retain RainSum 0`: this portion of the code initializes a new variable called `RainSum` with the initial value `0`. `Retain` ensures that the value is saved across rows rather than being reset for each time the PDV resets.
- `RainSum = RainSum + Precipitation` adds the current value of `Precipitation` to `RainSum`
:::

Let's try creating an accumulating column by groups. First, let's modify the code so we can create a new column that gives us the Day of the Week based off of the Date provided.

In [100]:
data temp_modified;
    set temp_modified;
    Day = WeekDay(Date);
run;

proc print data=temp_modified (obs=10);
run;

Obs,Date,Maximum,Minimum,Average,Departure,HDD,CDD,Precipitation,New Snow,Snow Depth,RainSum,Day
1,2024-09-01,88,77,82.5,7,0,18,0.07,0,0,0.07,1
2,2024-09-02,88,78,83,12,0,18,0.01,0,0,0.08,2
3,2024-09-03,89,77,83,12,0,18,0.21,0,0,0.29,3
4,2024-09-04,87,78,82.5,0.7,0,18,0.25,0,0,0.54,4
5,2024-09-05,81,76,78.5,-3.3,0,14,41,0,0,41.54,5
6,2024-09-06,89,78,83.5,17,0,19,0.08,0,0,41.62,6
7,2024-09-07,90,78,84,22,0,19,0.07,0,0,41.69,7
8,2024-09-08,89,77,83,13,0,18,0.07,0,0,41.76,1
9,2024-09-09,88,75,81.5,0.2,0,17,0.93,0,0,42.69,2
10,2024-09-10,87,75,81,-0.7,0,16,1.25,0,0,43.94,3


In order to get a cumualative sum of `Precipitation` by `Day`, we first need to sort the data by the group we would like.

In [102]:
proc sort data=temp_modified out=temp_modified_sort;
 BY Day;
run;

proc print data=temp_modified_sort (obs=5);
run;

Obs,Date,Maximum,Minimum,Average,Departure,HDD,CDD,Precipitation,New Snow,Snow Depth,RainSum,Day
1,2024-09-01,88,77,82.5,7,0,18,0.07,0,0,0.07,1
2,2024-09-08,89,77,83,13,0,18,0.07,0,0,41.76,1
3,2024-09-15,85,76,80.5,-12,0,16,0.63,0,0,54.48,1
4,2024-09-22,89,78,83.5,18,0,19,0.12,0,0,61.42,1
5,2024-09-29,88,78,83,12,0,18,0.02,0,0,.,1


In [104]:
data temp_modified_sort;
    set temp_modified_sort;
    keep Day Date Precipitation RainWeekDaySum;
    BY Day;
    retain RainWeekDaySum 0;
    if First.Day = 1 then RainWeekDaySum = 0;
    RainWeekDaySum+Precipitation;
run;

proc print data=temp_modified_sort;
run;

Obs,Date,Precipitation,Day,RainWeekDaySum
1,2024-09-01,0.07,1,0.07
2,2024-09-08,0.07,1,0.14
3,2024-09-15,0.63,1,0.77
4,2024-09-22,0.12,1,0.89
5,2024-09-29,0.02,1,0.91
6,2024-09-02,0.01,2,0.01
7,2024-09-09,0.93,2,0.94
8,2024-09-16,5.62,2,6.56
9,2024-09-23,1.8,2,8.36
10,2024-09-30,0.01,2,8.37


So why does the above code work? When we use the `BY` statement in a `DATA` step, it creates two temporary variables in our PDV for each BY variable.

- `FIRST.<variable>:` A flag indicating the first occurence of each new group for the specified variable.
- `LAST.<variable>:` A flag indicating the last occurence of each group for that variable.

These temporary variables do not get outputted to our final dataset. But we can reference it to separate our analysis by groups. It is important to remember that our data must be sorted by the `BY` variables in order for this to work!le.